In [9]:
import subprocess

work_dir = "."

proc = subprocess.run(
    ["HParse", "grammar", "wordnet"],
    cwd=work_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True  # return stdout/stderr as strings
)

# print the tool’s output or error
if proc.returncode == 0:
    print('all good???')
    print(proc.stdout)
else:
    print(f"✗ HParse failed (exit code {proc.returncode}):\n{proc.stderr}")

all good???



In [12]:
proc = subprocess.run(
    [
        "HDMan",
        "-m",                # merge
        "-w", "wlist",       # output word list
        "-n", "models0",     # output monophone list
        "-l", "dlog",        # logging file
        "dict",              # pronunciation dictionary
        "lexicon"            # lexicon name (without extension)
    ],
    cwd=work_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
if proc.returncode == 0:
    print("HDMan output:\n", proc.stdout)
else:
    print(f"✗ HDMan failed (exit {proc.returncode}):\n{proc.stderr}")

HDMan output:
 


In [13]:
proc = subprocess.run(
    [
        "HCopy",
        "-T", "1",                       # debug trace level
        "-C", "ParamConfig",             # feature extraction config
        "-S", "param.list"               # script file listing wav->mfcc mappings
    ],
    cwd=work_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

if proc.returncode == 0:
    print("HCopy output:\n", proc.stdout)
else:
    print(f"✗ HCopy failed (exit {proc.returncode}):\n{proc.stderr}")

HCopy output:
 data/2501/wINTERVALU_p2501_s1.wav -> data/2501/wINTERVALU_p2501_s1.mfcc 
data/2501/wV_p2501_s1.wav -> data/2501/wV_p2501_s1.mfcc 
data/2501/wPOSLEDNI_p2501_s1.wav -> data/2501/wPOSLEDNI_p2501_s1.mfcc 
data/2501/wHODIN_p2501_s1.wav -> data/2501/wHODIN_p2501_s1.mfcc 
data/2501/wDNU_p2501_s1.wav -> data/2501/wDNU_p2501_s1.mfcc 
data/2501/wTYDEN_p2501_s1.wav -> data/2501/wTYDEN_p2501_s1.mfcc 
data/2501/wTESLA_p2501_s1.wav -> data/2501/wTESLA_p2501_s1.mfcc 
data/2501/wZA_p2501_s1.wav -> data/2501/wZA_p2501_s1.mfcc 
data/2501/wAPPLE_p2501_s1.wav -> data/2501/wAPPLE_p2501_s1.mfcc 
data/2501/wROK_p2501_s1.wav -> data/2501/wROK_p2501_s1.mfcc 
data/2501/wZOBRAZ_p2501_s1.wav -> data/2501/wZOBRAZ_p2501_s1.mfcc 
data/2501/wMESIC_p2501_s1.wav -> data/2501/wMESIC_p2501_s1.mfcc 



In [26]:
# 1) build 8‐state prototype
proc8 = subprocess.run(
    [
        "HCompV",
        "-C", "TrainConfig",
        "-f", "0.01",
        "-m",
        "-S", "train.scp",
        "-M", "hmm0",
        "proto-8s-39f"
    ],
    cwd=work_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

if proc8.returncode == 0:
    print("✔ HCompV proto-8s-39f output:\n", proc8.stdout)
else:
    print(f"✗ HCompV proto-8s-39f failed (exit {proc8.returncode}):\n{proc8.stderr}")

# 2) build 3‐state prototype
proc3 = subprocess.run(
    [
        "HCompV",
        "-C", "TrainConfig",
        "-f", "0.01",
        "-m",
        "-S", "train.scp",
        "-M", "hmm0",
        "proto-3s-39f"
    ],
    cwd=work_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# the flags that never change
common = [
    "HERest",
    "-C", "TrainConfig",
    "-I", "train.mlf",
    "-t", "250.0", "150.0", "1000.0",
    "-S", "train.scp"
]

for i in range(0,6):
    in_hmm  = f"hmm{i}/hmmdefs"
    out_hmm = f"hmm{i+1}"
    cmd = common + ["-H", in_hmm, "-M", out_hmm, "models0"]

    print(f"\n>>> Pass {i+1}: {' '.join(cmd)}")
    proc = subprocess.run(
        cmd,
        cwd=work_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    if proc.returncode == 0:
        print(f"✔ Pass {i+1} succeeded:\n{proc.stdout}")
    else:
        print(f"✗ Pass {i+1} failed (exit {proc.returncode}):\n{proc.stderr}")
        break
else:
    print("\n✅ All 6 HERest passes completed successfully.")

✔ HCompV proto-8s-39f output:
 

>>> Pass 1: HERest -C TrainConfig -I train.mlf -t 250.0 150.0 1000.0 -S train.scp -H hmm0/hmmdefs -M hmm1 models0
✔ Pass 1 succeeded:
Pruning-On[250.0 150.0 1000.0]
 WARNING [-2331]  UpdateModels: V[1] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: ZA[2] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: TYDEN[3] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: ZOBRAZ[4] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: DNU[5] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: ROK[6] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: INTERVALU[8] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: HODIN[9] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: POSLEDNI[10] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: TESLA[11] copied: only 1 egs
 in HERest
 WARNING [-2331]  UpdateModels: APPLE[12] copied: only 1 egs
 in HERest
 WARNING [-2331]  Upd

In [ ]:
# build the exact HResults command you want
# dodod
cmd = [
    "HResults",
    "-e", "???", "SENT-START",
    "-e", "???",   "SENT-END",
    "-t",                          # show detailed timing
    "-I", "testref.mlf",           # reference transcription
    "models0",                     # model directory
    "recout.mlf"                   # recognizer output
]

print("\n>>> Running HResults:", " ".join(cmd))
proc = subprocess.run(
    cmd,
    cwd=work_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

if proc.returncode == 0:
    print("✔ HResults succeeded:\n", proc.stdout)
else:
    print(f"✗ HResults failed (exit {proc.returncode}):\n{proc.stderr}")